In [1]:
from google.colab import files
uploaded = files.upload()

Saving 서울특별시 광진구 주정차 단속 현황.csv to 서울특별시 광진구 주정차 단속 현황.csv


In [17]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
import pandas as pd
import glob
import matplotlib.colors as mcolors

import folium
from folium import plugins

import geopy
from geopy.geocoders import Nominatim

# 미사용
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import plotly.graph_objects as go
from datetime import datetime
from plotnine import *
from pandas.tseries.offsets import YearEnd
from sklearn.linear_model import LinearRegression

한글 폰트 깨짐 현상 해결 (둘 중 하나 사용)


In [ ]:
# 한글 폰트 깨짐 현상 해결 (실행 후 > 런타임 > 런타임 다시 시작 > 그래도 안되면 다시 실행)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
plt.rc('font', family = 'NanumBarunGothic')

In [ ]:
# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')

# csv 읽기

In [7]:
parking_crackdown_df = pd.read_csv('서울특별시 광진구 주정차 단속 현황.csv', encoding = 'cp949')

parking_crackdown_df.head()

,단속일시,단속원금,단속동,단속장소,위반내용,견인지시
0,2022-01-01 02:29:00,40000,중곡동,서울특별시 광진구 영화사로5길 19,교통소통장애,미견인
1,2022-01-01 06:41:00,80000,중곡동,150-196,소화전,미견인
2,2022-01-01 07:02:00,40000,구의동,서울 광진구 구의동 660-1 주차장앞,교통소통장애,미견인
3,2022-01-01 07:29:00,40000,구의동,서울 광진구 구의동 6-4,횡단보도,견인처리
4,2022-01-01 10:55:00,40000,구의동,리젠트오피스텔건너편,안전지대,미견인


필요한 데이터만 추출

In [8]:
parking = parking_crackdown_df[['단속일시', '단속동', '단속장소', '위반내용']]
parking

,단속일시,단속동,단속장소,위반내용
0,2022-01-01 02:29:00,중곡동,서울특별시 광진구 영화사로5길 19,교통소통장애
1,2022-01-01 06:41:00,중곡동,150-196,소화전
2,2022-01-01 07:02:00,구의동,서울 광진구 구의동 660-1 주차장앞,교통소통장애
3,2022-01-01 07:29:00,구의동,서울 광진구 구의동 6-4,횡단보도
4,2022-01-01 10:55:00,구의동,리젠트오피스텔건너편,안전지대
...,...,...,...,...
39727,2023-02-28 20:43:00,군자동,군자빌딩,주정차금지(황색실선)구역
39728,2023-02-28 21:23:00,자양동,서울 광진구 자양동 659-32,보도
39729,2023-02-28 21:38:00,화양동,서울 광진구 능동로 154,보도
39730,2023-02-28 22:36:00,자양동,서울 광진구 자양동 12-10,교차로


In [107]:
geolocator = Nominatim(user_agent = "specify_here")
location = geolocator.geocode("서울특별시 광진구 영화사로5길 19")

print(location.latitude, location.longitude)

37.5567054 127.0917041


location을 for문으로 받음

In [105]:
def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding("대구 태전동")
print(crd['lat'])
print(crd['lng'])

35.9234127
128.5459483


In [26]:
map = folium.Map(location=[37.5567054, 127.0917041], zoom_start=15)
map

In [99]:
geo_local = Nominatim(user_agent='South Korea')
lan_log = []

def geocoding(address):
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

parking_location = geocoding(parking)
lan_log.append(parking_location)

AttributeError: ignored